In [3]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [7]:
class HoNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = HoNet().to(device)
print(model)

HoNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [14]:
x = torch.rand(1, 28, 28, device=device)
logits = model(x)
pred_probs = nn.Softmax(dim=1)(logits)
y_pred = pred_probs.argmax(1)
print("Predicted class:", y_pred.cpu().item())

Predicted class: 9


### nn.Flatten

In [17]:
input_image = torch.rand(3, 28, 28)
input_image.size()

torch.Size([3, 28, 28])

In [22]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
flat_image.size()

torch.Size([3, 784])

In [31]:
# How about more dimension?
image3 = torch.rand(3, 28, 28, 28)
f3 = flatten(image3)
f3.size()

torch.Size([3, 21952])

### Observations

- `nn.Flatten()` leaves the first dimension (= batch?) and flattens all others

### nn.Linear

In [30]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
hidden1.size()

torch.Size([3, 20])

### nn.ReLU

In [27]:
x = torch.rand(10) - 0.5

In [28]:
relu = nn.ReLU()
rx = relu(x)
x, rx

(tensor([ 0.3427, -0.0370,  0.3672,  0.4891,  0.4188,  0.1606,  0.0590,  0.3264,
          0.0351,  0.0750]),
 tensor([0.3427, 0.0000, 0.3672, 0.4891, 0.4188, 0.1606, 0.0590, 0.3264, 0.0351,
         0.0750]))

### nn.Sequential

In [34]:
seq = nn.Sequential(
    flatten,
    layer1,
    relu,
    nn.Linear(20, 10)
)

In [35]:
input_image = torch.rand(3, 28, 28)
logits = seq(input_image)
logits.size()

torch.Size([3, 10])

### nn.Softmax

In [39]:
softmax = nn.Softmax(dim=1)
pred_probs = softmax(logits)
pred_probs.size()

torch.Size([3, 10])

In [41]:
logits[0, :4], \
pred_probs[0, :4]

(tensor([ 0.1962, -0.2523,  0.0532, -0.0633], grad_fn=<SliceBackward>),
 tensor([0.1152, 0.0736, 0.0998, 0.0889], grad_fn=<SliceBackward>))

### Model Parameters

In [65]:
params = model.named_parameters()

In [66]:
for key, value in params:
    print("#" * 80)
    print(key)
    print(value[:2])

################################################################################
linear_relu_stack.0.weight
tensor([[ 0.0022, -0.0252, -0.0065,  ..., -0.0014, -0.0214,  0.0197],
        [ 0.0035,  0.0111,  0.0084,  ...,  0.0179, -0.0266, -0.0157]],
       device='cuda:0', grad_fn=<SliceBackward>)
################################################################################
linear_relu_stack.0.bias
tensor([ 0.0277, -0.0018], device='cuda:0', grad_fn=<SliceBackward>)
################################################################################
linear_relu_stack.2.weight
tensor([[ 0.0166,  0.0327,  0.0410,  ...,  0.0282, -0.0258,  0.0229],
        [-0.0160,  0.0062,  0.0177,  ...,  0.0111,  0.0378,  0.0217]],
       device='cuda:0', grad_fn=<SliceBackward>)
################################################################################
linear_relu_stack.2.bias
tensor([0.0138, 0.0223], device='cuda:0', grad_fn=<SliceBackward>)
########################################################

### State Dict

Seems to only give raw data, not live references.

In [74]:
for key, value in model.state_dict().items():
    print("#" * 80)
    print(key)
    print(value[:2])

################################################################################
linear_relu_stack.0.weight
tensor([[ 0.0022, -0.0252, -0.0065,  ..., -0.0014, -0.0214,  0.0197],
        [ 0.0035,  0.0111,  0.0084,  ...,  0.0179, -0.0266, -0.0157]],
       device='cuda:0')
################################################################################
linear_relu_stack.0.bias
tensor([ 0.0277, -0.0018], device='cuda:0')
################################################################################
linear_relu_stack.2.weight
tensor([[ 0.0166,  0.0327,  0.0410,  ...,  0.0282, -0.0258,  0.0229],
        [-0.0160,  0.0062,  0.0177,  ...,  0.0111,  0.0378,  0.0217]],
       device='cuda:0')
################################################################################
linear_relu_stack.2.bias
tensor([0.0138, 0.0223], device='cuda:0')
################################################################################
linear_relu_stack.4.weight
tensor([[-0.0162, -0.0044,  0.0212,  ...,  0.036